In [65]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr
from sklearn.preprocessing import scale
from scipy.stats import t


In [66]:
rw_price = pd.read_csv("rw_price.csv")
rw_returns = pd.read_csv("rw_returns.csv")
log_rw_returns = pd.read_csv("log_rw_returns.csv")

In [67]:
winLen = 200
Nday = len(log_rw_returns)
print(Nday)

S_cov = np.corrcoef(scale(log_rw_returns).T)
S_cov

999


array([[1.        , 0.34982739, 0.32699292, 0.27549521, 0.34886486],
       [0.34982739, 1.        , 0.37785361, 0.33174046, 0.32484628],
       [0.32699292, 0.37785361, 1.        , 0.33200448, 0.34886946],
       [0.27549521, 0.33174046, 0.33200448, 1.        , 0.32947489],
       [0.34886486, 0.32484628, 0.34886946, 0.32947489, 1.        ]])

### Checking Graph operators from 
A Unified Framework for Structured Graph Learning via Spectral Constraints

https://jmlr.org/papers/volume21/19-276/19-276.pdf

In [68]:
def w_from_L(M: np.ndarray) -> np.ndarray:
    M = np.asarray(M)
    if M.ndim != 2 or M.shape[0] != M.shape[1]:
        raise ValueError("M must be a square matrix")

    n = M.shape[0]
    w = np.empty(n * (n - 1) // 2, dtype=M.dtype)
    k = 0
    for i in range(n - 1):            # i = 0 .. n-2
        for j in range(i + 1, n):     # j = i+1 .. n-1 (strict upper)
            w[k] = -M[i, j]
            k += 1
    return w

def L_from_w(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    k = w.size
    n = int((1 + np.sqrt(1 + 8 * k)) / 2)
    if n * (n - 1) // 2 != k:
        raise ValueError("Invalid length of w: must be n(n-1)/2 for some integer n.")

    Lw = np.zeros((n, n), dtype=float)

    # Fill the strict upper triangle row by row, backwards like in C++
    for i in range(n - 2, -1, -1):
        j = n - i - 1
        # Equivalent of Eigen: Lw.row(i).tail(j) = -w.head(k).tail(j)
        Lw[i, -j:] = -w[:k][-j:]
        k -= j

    # Make symmetric: copy upper triangle into lower
    Lw = Lw + Lw.T

    # Adjust diagonal: subtract column sums
    colsum = Lw.sum(axis=0)
    np.fill_diagonal(Lw, Lw.diagonal() - colsum)

    return Lw

def W_init(M):
    W0 = w_from_L(M)
    W0[W0 < 0] = 0
    return W0

def L_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Lstar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Lstar[k-1] = Y[i-1,i-1] - Y[i-1, j-1] - Y[ j-1, i-1] + Y[ j-1, j-1]


    return Lstar

def A_from_w(w):

    w = np.asarray(w, dtype=float).ravel()
    k = w.size

    # Infer n from k = n*(n-1)/2
    n_float = (1 + np.sqrt(1 + 8*k)) / 2
    n = int(n_float)
    if n*(n-1)//2 != k:
        raise ValueError(f"len(w)={k} is not a triangular number; expected k=n*(n-1)/2.")

    A = np.zeros((n, n), dtype=float)

    # Indices of strict upper triangle in row-major order
    iu = np.triu_indices(n, k=1)
    A[iu] = w            # fill upper triangle
    A[(iu[1], iu[0])] = w  # mirror to lower triangle
    return A

def w_from_A(M):
    
    N = M.shape[1]
    k = N * (N - 1) // 2
    w = np.zeros(k)
    l = 0

    for i in range(N - 1):
        for j in range(i + 1, N):
            w[l] = M[i, j]
            l += 1
    return w


def A_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Astar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Astar[k-1] = Y[i-1, j-1] + Y[ j-1, i-1]


    return Astar

def D_star(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    dStar = L_star(np.diag(w))
    return  dStar

In [69]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

S_cov = np.corrcoef(scale(log_rw_returns).T)
print("S_cov")
print(S_cov)

S_prec = np.linalg.pinv(S_cov)
print("S_prec")
print(S_prec)

S_cov
[[1.         0.34982739 0.32699292 0.27549521 0.34886486]
 [0.34982739 1.         0.37785361 0.33174046 0.32484628]
 [0.32699292 0.37785361 1.         0.33200448 0.34886946]
 [0.27549521 0.33174046 0.33200448 1.         0.32947489]
 [0.34886486 0.32484628 0.34886946 0.32947489 1.        ]]
S_prec
[[ 1.27263721 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  1.32359962 -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  1.32304704 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  1.25090233 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  1.29921337]]


In [70]:
wL = w_from_L(S_prec)
print("w from L")
print(wL)

w = W_init(S_prec)
print("w")
print(w)

Lw = L_from_w(w)
print("Lw from w")
print(Lw) 

Lstar = L_star(Lw)
print("Lstar from Lw")
print(Lstar) 

Aw = A_from_w(w)
print('Aw from w')
print(Aw)

wA = w_from_A(Aw)
print('w from A')
print(wA)

Astar = A_star(Aw)
print('Astar from Aw')
print(Astar)

Dstar = D_star(w)
print('Dstar from w')
print(Dstar)

w from L
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
w
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
Lw from w
[[ 0.81800315 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  0.9256273  -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  0.92470283 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  0.77792007 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  0.88190495]]
Lstar from Lw
[2.24258162 2.12834103 1.83466448 2.21258694 2.42273564 2.14160172
 2.14937582 2.13318397 2.26741184 2.10830848]
Aw from w
[[0.         0.24947558 0.19281752 0.11937063 0.25633942]
 [0.24947558 0.         0.28620275 0.21902718 0.17092178]
 [0.19281752 0.28620275 0.         0.21528053 0.23040203]
 [0.11937063 0.21902718 0.21528053 0.         0.22424173]
 [0.25633942 0.17092178 0.23040203 0.22424173 0.        ]]
w from A


## Code From Paper

In [71]:
def L_update( wn, Phi_n, rho, p, k):

    ''' 
    L_n: Laplace Matrix (Ln = Diag(Wn1) - Wn = Lwn) Laplacian operator “A Unified Framework for Structured Graph Learning via Spectral Constraints”
    Phi_n: ADMM Dual Variable 
    rho: ADMM Hyperparameter
    p: Number of time series
    k: Clusters
    '''
    # Reference: Equation 16
    
    Y = L_from_w(wn) + Phi_n/rho

    eigVals,eigVecs = np.linalg.eigh(Y)
    U = eigVecs[:,p-k:]
    R = np.diag(eigVals[p-k:])
    I = np.eye(R.shape[0]) 
    Ln = 0.5 * U @ ( R + np.sqrt(  R**2 + 4/rho * I  ) ) @ U.T
    return Ln

p = Lw.shape[0]
Phi_n = np.zeros((p,p)) # ADMM dual variable for Laplace Matrix
rho = 1                 # ADMM hyperparameter.
k = 1                   # Clusters


Ln = L_update( w, Phi_n, rho, p, k)
print(Ln)

[[ 0.16414457 -0.3585202   0.31150615  0.07536903 -0.19249955]
 [-0.3585202   0.7830703  -0.68038345 -0.16461904  0.4204524 ]
 [ 0.31150615 -0.68038345  0.5911623   0.14303195 -0.36531695]
 [ 0.07536903 -0.16461904  0.14303195  0.03460664 -0.08838858]
 [-0.19249955  0.4204524  -0.36531695 -0.08838858  0.22575268]]


In [73]:
from scipy.stats import multivariate_t

# mean (mu)
loc = [0., 0., 0., 0., 0.]

# covariance (Sigma)
shape = [
    [1.5,     0.75,    0.375,   0.1875,  0.09375],
    [0.75,    1.5,     0.75,    0.375,   0.1875 ],
    [0.375,   0.75,    1.5,     0.75,    0.375  ],
    [0.1875,  0.375,   0.75,    1.5,     0.75   ],
    [0.09375, 0.1875,  0.375,   0.75,    1.5    ],
]

# degrees of freedom
df = 6.0

# Create a frozen multivariate_t object
# This allows fixing the parameters and then calling methods like rvs()
dist = multivariate_t(loc=loc, shape=shape, df=df)

# Generate samples
num_samples = 1000
samples = dist.rvs(size=num_samples)
samples.shape 

(1000, 5)

In [81]:
import numpy as np
from numpy.linalg import slogdet
from scipy.special import digamma, polygamma, gammaln
# Reference
# https://zouyuxin.github.io/Note/EMtDistribution.pdf
# https://shoichimidorikawa.github.io/Lec/ProbDistr/t-e.pdf
def fit_multivariate_t(
    X,
    nu_init=10.0,
    max_iter=200,
    tol=1e-6,
    fix_nu=None,
    jitter=1e-6,
    verbose=False,
):
    """
    Estimate (mu, Sigma, nu) for a multivariate Student's t via EM.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Data.
    nu_init : float
        Initial degrees of freedom (ignored if fix_nu is not None).
    max_iter : int
        Maximum EM iterations.
    tol : float
        Relative tolerance on log-likelihood for convergence.
    fix_nu : float or None
        If set, keep nu fixed at this value.
    jitter : float
        Diagonal jitter multiplier for numerical stability.
    verbose : bool
        Print progress if True.

    Returns
    -------
    mu : array, shape (p,)
    Sigma : array, shape (p, p)
    nu : float
    history : dict with 'loglik'
    """
    X = np.asarray(X)
    n, p = X.shape

    # Initialize
    mu = X.mean(axis=0)
    # sample covariance (unbiased=False), add tiny jitter
    centered0 = X - mu
    Sigma = centered0.T @ centered0 / n
    Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)
    nu = float(nu_init if fix_nu is None else fix_nu)

    def mahal_sq(Sigma, X, mu):
        # δ_i = (x_i - mu)^T Sigma^{-1} (x_i - mu)
        L = np.linalg.cholesky(Sigma)
        Y = np.linalg.solve(L, (X - mu).T)   # shape (p, n)
        return np.sum(Y * Y, axis=0)         # shape (n,)

    def loglik(mu, Sigma, nu):
        # log-likelihood of multivariate t
        sgn, logdet = slogdet(Sigma)
        if sgn <= 0:
            return -np.inf
        delta = mahal_sq(Sigma, X, mu)
        term1 = gammaln((nu + p) / 2) - gammaln(nu / 2)
        term2 = - (p / 2) * np.log(nu * np.pi) - 0.5 * logdet
        term3 = - ((nu + p) / 2) * np.log1p(delta / nu)
        return np.sum(term1 + term2 + term3)

    ll_old = -np.inf
    history = {'loglik': []}

    for it in range(1, max_iter + 1):
        # E-step
        delta = mahal_sq(Sigma, X, mu)
        w = (nu + p) / (nu + delta)  # E[lambda_i | x_i]
        eloglam = digamma((nu + p) / 2) - np.log((nu + delta) / 2)  # E[log lambda_i | x_i]

        # M-step: mu (weighted mean)
        sumw = w.sum()
        mu = (w[:, None] * X).sum(axis=0) / sumw

        # M-step: Sigma
        Xc = X - mu
        Sigma = (Xc.T * w) @ Xc / n
        # stabilize
        Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)

        # M-step: nu (solve for root of dQ/dnu = 0) unless fixed
        if fix_nu is None:
            # f(nu) = log(nu/2) - psi(nu/2) + 1 + (1/n) * sum(E[log lambda_i] - E[lambda_i]) = 0
            c = (eloglam.mean() - w.mean())

            def f(nu_):
                return np.log(nu_ / 2.0) - digamma(nu_ / 2.0) + 1.0 + c

            def fprime(nu_):
                return 1.0 / nu_ - 0.5 * polygamma(1, nu_ / 2.0)

            # Newton update with simple guarding
            nu_new = max(nu, 2.01)  # keep > 2 so covariance exists
            for _ in range(30):
                val = f(nu_new)
                der = fprime(nu_new)
                step = val / der
                nu_new = nu_new - step
                if nu_new < 2.01:
                    nu_new = 2.01
                if abs(step) / nu_new < 1e-6:
                    break
            nu = float(nu_new)
        else:
            nu = float(fix_nu)

        # Evaluate log-likelihood and check convergence
        ll = loglik(mu, Sigma, nu)
        history['loglik'].append(ll)
        if verbose:
            print(f"iter {it:3d}: ll={ll:.6f}, nu={nu:.4f}")

        if np.isfinite(ll_old):
            if abs(ll - ll_old) / (abs(ll_old) + 1e-12) < tol:
                break
        ll_old = ll

    return mu, Sigma, nu, history


# X: (n_samples, n_features) data

mu, Sigma, nu, history = fit_multivariate_t(
samples,
nu_init=30.0,
max_iter=200,
tol=1e-8,
fix_nu=None,
jitter=1e-6,
verbose=False,
)
print("mu:", mu)
print("nu:", nu)
print("Sigma:\n", Sigma)

mu: [-0.01214302 -0.04084377 -0.02271728  0.01501869  0.03775479]
nu: 6.376636097962273
Sigma:
 [[1.50124719 0.71356759 0.3875365  0.18888086 0.10600994]
 [0.71356759 1.49597974 0.72514147 0.38102188 0.15437081]
 [0.3875365  0.72514147 1.56357744 0.6987182  0.34110254]
 [0.18888086 0.38102188 0.6987182  1.49837025 0.7529836 ]
 [0.10600994 0.15437081 0.34110254 0.7529836  1.55807296]]


In [ ]:
def w_update( x, beta, rho, a, w_n_prev, w_n, L_n, Vn, p, k, d, Tn, nu, eta, Phi_n, mu_n, z_n):

    ''' 
    x: observation for each time series
    p: Number of Time Series
    a: VAR coefficients
    k: Number of Clusters
    nu: Student T degrees of freedom
    Vn: Laplacian spectral constraint
    Tn: Number of observations in a Frame 
    eta: regularization term
    d: Node degree constraint

    Phi_n: ADMM dual variable
    mu_n: ADMM dual variable
    z_n: ADMM dual variable
    '''

    assert Vn.shape[0] == p and Vn.shape[1] == k

    Lstar = L_star(w) 
    dstar = D_star(w)

    u_n = w_n - a * w_n_prev # Vector Autoregressive variable

    cth = np.sqrt( (2*beta)/(rho*(4*rho -1)) ) * np.ones(len(w_n))

    S_sum = 0
    for i in range(0,Tn):
        S_sum += ( x[i,:]*x[i,:].T )/( x[i,:].T @ L_n @ x[i,:] + nu )

    S = ( (p + nu)/Tn ) * S_sum

    aw = L_star @ ( S + Phi_n + rho*(L_star - L_n) + eta * Vn@Vn.T )

    bw = -mu_n - rho*( u_n + a*w_n_prev ) + d_star @ ( z_n - rho * ( d - d_star*w_n ) )

    cw = (1 - (rho)/(rho*(4*p-1))) * wn - (1)/( rho*(4*p -1) ) * (aw + bw)

    wn = (cw > cth) * cw

    return wn